# Discover Spotify
To use this file, you first need to manually execute the file 'get_token.py' and follow the instructions in the console.

The resulting token was saved in 'token.txt' and is valid for one hour.

If a valid token has been stored, the following code can be executed.

# Authorize

In [ ]:
import spotipy
import json
import random

with open('token.txt', 'r') as f:
    token = f.read()

sp = spotipy.Spotify(auth=token)

In [ ]:
# Get the current user
# (This is needed to create a playlist later on)

USER = sp.current_user()
USER_ID = USER['id']

# Discover

In [ ]:
def print_top_items(top_items, item_type):
    print(f'\n\nYour top {item_type} are:')
    for i, item in enumerate(top_items):
        item_name = item['name']
        print(f'{i+1}. {item_name}')

In [ ]:
def save_dict_to_json(d, filename):
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(d, f, ensure_ascii=False, indent=4)

### Top artists and tracks

In [ ]:
# Get the user's top artists
top_artists = sp.current_user_top_artists(limit=50, time_range='medium_term')

# Get the user's top tracks
top_tracks = sp.current_user_top_tracks(limit=50, time_range='medium_term')

# Get the user's top genres
top_genres = top_artists['items'][0]['genres']

In [ ]:
print_top_items(top_artists['items'][0:10], 'artists')
print_top_items(top_tracks['items'][0:10], 'tracks')

save_dict_to_json(top_artists, 'top_artists.json')
save_dict_to_json(top_tracks, 'top_tracks.json')

### Recommendations

Get recommendations based on the user's top artists, genres, and tracks

In [ ]:
attributes = {
    # 'min_acousticness': -1, # 0 = not acoustic, 1 = acoustic
    # 'max_acousticness': -1, # 0 = not acoustic, 1 = acoustic
    # 'target_acousticness': -1, # 0 = not acoustic, 1 = acoustic
    # 'min_danceability': -1, # 0 = not danceable, 1 = danceable
    # 'max_danceability': -1, # 0 = not danceable, 1 = danceable
    # 'target_danceability': -1, # 0 = not danceable, 1 = danceable
    # 'min_duration_ms': -1, # in milliseconds
    # 'max_duration_ms': -1, # in milliseconds
    # 'target_duration_ms': -1, # in milliseconds
    # 'min_energy': -1, # 0 = not energetic, 1 = energetic
    # 'max_energy': -1, # 0 = not energetic, 1 = energetic
    # 'target_energy': -1, # 0 = not energetic, 1 = energetic
    # 'min_instrumentalness': -1, # 0 = not instrumental, 1 = instrumental
    # 'max_instrumentalness': -1, # 0 = not instrumental, 1 = instrumental
    # 'target_instrumentalness': -1, # 0 = not instrumental, 1 = instrumental
    # 'min_key': -1, # -1 = Not detected, 0 = C, 1 = C#, 2 = D, ..., 11 = B
    # 'max_key': -1, # -1 = Not detected, 0 = C, 1 = C#, 2 = D, ..., 11 = B
    # 'target_key': -1, # -1 = Not detected, 0 = C, 1 = C#, 2 = D, ..., 11 = B
    # 'min_liveness': -1, # 0 = not live, 1 = live
    # 'max_liveness': -1, # 0 = not live, 1 = live
    # 'target_liveness': -1, # 0 = not live, 1 = live
    # 'min_loudness': -1, # in dB, -60 = quietest, 0 = loudest
    # 'max_loudness': -1, # in dB, -60 = quietest, 0 = loudest
    # 'target_loudness': -1, # in dB, -60 = quietest, 0 = loudest
    # 'min_mode': -1, # 0 = minor, 1 = major
    # 'max_mode': -1, # 0 = minor, 1 = major
    # 'target_mode': -1, # 0 = minor, 1 = major
    # 'min_popularity': -1, # 0 = least popular, 100 = most popular
    # 'max_popularity': -1, # 0 = least popular, 100 = most popular
    # 'target_popularity': -1, # 0 = least popular, 100 = most popular
    # 'min_speechiness': -1, # 0 = least speechy, 1 = most speechy
    # 'max_speechiness': -1, # 0 = least speechy, 1 = most speechy
    # 'target_speechiness': -1, # 0 = least speechy, 1 = most speechy
    # 'min_tempo': -1, # in BPM
    # 'max_tempo': -1, # in BPM
    # 'target_tempo': -1, # in BPM
    # 'min_time_signature': -1, # 3 = 3/4, 4 = 4/4, 5 = 5/4, 6 = 6/4, 7 = 7/4
    # 'max_time_signature': -1, # 3 = 3/4, 4 = 4/4, 5 = 5/4, 6 = 6/4, 7 = 7/4
    # 'target_time_signature': -1, # 3 = 3/4, 4 = 4/4, 5 = 5/4, 6 = 6/4, 7 = 7/4
    # 'min_valence': -1, # 0 = sad, 1 = happy
    # 'max_valence': -1, # 0 = sad, 1 = happy
    # 'target_valence': -1 # 0 = sad, 1 = happy
}

# attributes['min_acousticness'] = random.uniform(0, 1)
# attributes['max_acousticness'] = random.uniform(attributes['min_acousticness'], 1)
# attributes['target_acousticness'] = random.uniform(attributes['min_acousticness'], attributes['max_acousticness'])

# attributes['min_danceability'] = random.uniform(0, 1)
# attributes['max_danceability'] = random.uniform(attributes['min_danceability'], 1)
# attributes['target_danceability'] = random.uniform(attributes['min_danceability'], attributes['max_danceability'])

# attributes['min_duration_ms'] = random.randint(0, 600000)
# attributes['max_duration_ms'] = random.randint(attributes['min_duration_ms'], 600000)
# attributes['target_duration_ms'] = random.randint(attributes['min_duration_ms'], attributes['max_duration_ms'])

# attributes['min_energy'] = random.uniform(0, 1)
# attributes['max_energy'] = random.uniform(attributes['min_energy'], 1)
# attributes['target_energy'] = random.uniform(attributes['min_energy'], attributes['max_energy'])

# attributes['min_instrumentalness'] = random.uniform(0, 1)
# attributes['max_instrumentalness'] = random.uniform(attributes['min_instrumentalness'], 1)
# attributes['target_instrumentalness'] = random.uniform(attributes['min_instrumentalness'], attributes['max_instrumentalness'])

# attributes['min_key'] = random.randint(-1, 11)
# attributes['max_key'] = random.randint(attributes['min_key'], 11)
# attributes['target_key'] = random.randint(attributes['min_key'], attributes['max_key'])

# attributes['min_liveness'] = random.uniform(0, 1)
# attributes['max_liveness'] = random.uniform(attributes['min_liveness'], 1)
# attributes['target_liveness'] = random.uniform(attributes['min_liveness'], attributes['max_liveness'])

# attributes['min_loudness'] = random.uniform(-60, 0)
# attributes['max_loudness'] = random.uniform(attributes['min_loudness'], 0)
# attributes['target_loudness'] = random.uniform(attributes['min_loudness'], attributes['max_loudness'])

# attributes['min_mode'] = random.randint(0, 1)
# attributes['max_mode'] = random.randint(attributes['min_mode'], 1)
# attributes['target_mode'] = random.randint(attributes['min_mode'], attributes['max_mode'])

# attributes['min_popularity'] = random.randint(0, 100)
# attributes['max_popularity'] = random.randint(attributes['min_popularity'], 100)
# attributes['target_popularity'] = random.randint(attributes['min_popularity'], attributes['max_popularity'])

# attributes['min_speechiness'] = random.uniform(0, 1)
# attributes['max_speechiness'] = random.uniform(attributes['min_speechiness'], 1)
# attributes['target_speechiness'] = random.uniform(attributes['min_speechiness'], attributes['max_speechiness'])

# attributes['min_tempo'] = random.uniform(0, 250)
# attributes['max_tempo'] = random.uniform(attributes['min_tempo'], 250)
# attributes['target_tempo'] = random.uniform(attributes['min_tempo'], attributes['max_tempo'])

# attributes['min_time_signature'] = random.randint(3, 7)
# attributes['max_time_signature'] = random.randint(attributes['min_time_signature'], 7)
# attributes['target_time_signature'] = random.randint(attributes['min_time_signature'], attributes['max_time_signature'])

# attributes['min_valence'] = random.uniform(0, 1)
# attributes['max_valence'] = random.uniform(attributes['min_valence'], 1)
# attributes['target_valence'] = random.uniform(attributes['min_valence'], attributes['max_valence'])

In [ ]:
# round values to 2 decimal places
for key in attributes:
    if attributes[key] != -1:
        attributes[key] = round(attributes[key], 2)

seed_artists = [artist['id'] for artist in top_artists['items']][0:2]
seed_genres = []
for artist in top_artists['items'][2:]:
    for genre in artist['genres']:
        if genre not in seed_genres:
            seed_genres.append(genre)
seed_genres = seed_genres[0:2]
seed_tracks = [track['id'] for track in top_tracks['items']][0:1]

print(seed_artists)
print(seed_genres)
print(seed_tracks)

recommendations = sp.recommendations(seed_artists=seed_artists,
                                     seed_genres=seed_genres,
                                     seed_tracks=seed_tracks,
                                     **attributes,
                                     limit=100)

Save recommendations to a new playlist

In [ ]:
playlist = sp.user_playlist_create(user=USER_ID, name='Test Random', public=False, collaborative=False, description='Test')
sp.user_playlist_add_tracks(user=USER_ID, playlist_id=playlist['id'], tracks=[track['uri'] for track in recommendations['tracks']])